## Review: what we did in Week 3: Amazon data
* Read Amazon.csv
* Get to know the data
* Create a smaller subset of the data
## [Jump to Week 4 material](#thisWeek)

In [ ]:
# imports and specifications
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### read Amazon.csv

In [ ]:
amazon = pd.read_csv('/Users/Aaron_hill/Dropbox/data/Amazon.csv')

### get to know the data

In [ ]:
print("amazon is:", type(amazon))
print("amazon has", amazon.shape[0], "rows and", amazon.shape[1], "columns", "\n")
print("the data types for each of the columns in amazon:")
print(amazon.dtypes, "\n")
print("the first 10 rows in amazon:")
print(amazon.head(10))

### create a ndarray for `L`

In [ ]:
L = amazon["helpful"]
print(type(L))
print(type(L.values))
print(L.shape)

### create a ndarray for `X`
Use only "Score" and "Time" as features, for now.

In [ ]:
X = amazon[["Score", "Time"]]
print(type(X))
print(type(X.values))
print(X.shape)

## <a name='thisWeek'></a>Week 4: fit linear classifier using gradient descent and assess the fit of the model

### using the `SGDClassifier` class in `linear_model`, fit the model according to given training data

In [ ]:
from sklearn import linear_model
sgd = linear_model.SGDClassifier(loss="squared_loss")
sgd.fit(X, L)

In [ ]:
# number & proportion of accurate predictions
print(sum(sgd.predict(X) == L.values))
print(sum(sgd.predict(X) == L.values) / L.shape)

### how well did we do? compare the model's predictions for  `Y` to the labels `L`
We'll start with the first few measures in Flach, p. 57

In [ ]:
import my_measures

sgd_pm = my_measures.BinaryClassificationPerformance(sgd.predict(X), L, 'sgd')
sgd_pm.compute_measures()
print(sgd_pm.performance_measures)

## Normalization

*[Normalization](https://scikit-learn.org/stable/modules/preprocessing.html#normalization) is the process of scaling individual samples to have unit norm.*

In [ ]:
X.describe()

In [ ]:
from sklearn import preprocessing
X_normalized = preprocessing.normalize(X)

In [ ]:
pd.DataFrame(X_normalized).describe()